In [19]:
# GET JUST THE PREVIOUS MONTH'S data

import pandas as pd

df = pd.read_json('allData_19042021_export.json')
df_2021 = df.loc[df['year'] == 2021]
old_data_df = pd.read_csv("Cloud_SQL_Export_2021-04-12 (19_47_25).csv")
# print(len(old_data_df.index))

df_rows = []
indicators = ['field_ndvi', 'field_ndwi', 'field_rainfall', 'field_temperature']
for index, row in df_2021.iterrows():
    for indicator in indicators:
        df_row = {}
        row_2021 = df_2021.loc[df_2021['field_id'] == row['field_id']]
        df_row['field_id']= row['field_id']
        df_row['indicator'] = indicator
        for key, value in row[indicator].items():
        # for key, value in json.loads(row[indicator]).items():
            df_row[key] = value
        df_rows.append(df_row)
        
df2 = pd.DataFrame(df_rows)

for index, row in old_data_df.iterrows():
    try:
        value_ = df2["march"].loc[(df2["field_id"] == row["field_id"]) & (df2["indicator"] == row["indicator"])]
        # if not value_.empty:
        if value_.values[0]:
            old_data_df.loc[index, "march"] = float(value_.values[0])
            # row["march"] = float(value_.values[0])
    except IndexError:
        continue

old_data_df.to_csv("updated_march.csv", index=False)

1268      0.172
1269      0.131
1270           
1271    300.862
Name: april, dtype: object


In [1]:
# GET LAST 12 MONTH'S DATA
from tqdm import tqdm
import pandas as pd
import numpy as np
import json

# data = geopandas.read_file("crops.geojson")
# data.head() # show the first 5 rows of the data.

df = pd.read_json('farmafrica_data_can_finally_move_on.json')
df_2020 = df.loc[df['year'] == 2020]
df_2021 = df.loc[df['year'] == 2021]

df_rows = []
indicators = ['field_ndvi', 'field_ndwi', 'field_rainfall', 'field_temperature']
for index, row in tqdm(df_2020.iterrows()):
    for indicator in indicators:
        df_row = {}
        row_2021 = df_2021.loc[df_2021['field_id'] == row['field_id']]
        df_row['field_id']= row['field_id']
        # df_row['user_id']= row['user_id']
        df_row['indicator'] = indicator
        for key, value in row[indicator].items():
        # for key, value in json.loads(row[indicator]).items():
            df_row[key] = value
            if key in ["january", "february", "march"]:
                df_row[key] = row_2021[indicator].iloc[0][key]
            # if key == "january":
            #     df_row[key] = row_2021[indicator].iloc[0][key]
            # elif key == "february" and indicator != "field_rainfall":
            #     df_row[key] = row_2021[indicator].iloc[0][key]

        df_rows.append(df_row)

df2 = pd.DataFrame(df_rows)
print(len(df2.index))
# df2.replace('', np.nan, regex=True, inplace=True)
# df2.dropna(how='all', axis='columns', inplace=True)
# print(f"2021 data length is {len(df.index)}")
# df2.iloc[0]
# df2.iloc[0]['may']
# print(len(df2.index))
# df2.to_csv("jan_feb_data.csv", index=False)
df2.to_json('formatted_farmafrica_data_can_finally_move_on.json', orient='records')
# print("doneeeee")

14it [00:00, 171.51it/s]56



## Application Exercises

These exercises will be used to evaluate your ability to work with geospatial data in Python. You will need several python packages to complete these exercises, the packages required are as follows:

* geopandas
* pandas
* rasterio
* fiona
* ) == 1144636.0, "Sorry wrong answer"
# # assert str(reproject_raster('crops.tif', 'EPSG:4326').crs) == 'EPSG:4326', "Sorry wrong answer"
# print("Congratulations, all is working just fine !!!")

For each cell where you see "WRITE YOUR CODE HERE" delete the return notImplemented statement when you write your code there - don't leave it in the notebook. This notebook contains test cases to evaluate whether your functions are working correctly. When you run these cells they will print either "Congratulations, all is working just fine !!!" if your function works or "Sorry wrong answer" if it does not. Once you finish these exercises, upload the completed notebook to the application page and submit your application.

In case of any questions, contact the organizers at hello@radiant.earth

### Reading in data using GeoPandas
[GeoPandas](https://geopandas.org/) is a Python library that is particularly good for doing any geospatial-data-related tasks in Python. We will use several other libraries in Python as well.

To test that everything is working fine:
1. Ensure that all of the files from the repository are in the same directory as this notebook.
2. Use GeoPandas to read in the first sheet of data from the geojson file as shown below
3. Work through some of the example analyses shown and complete the questions that follow.

In [25]:
# FINDING AREA FOR FIELDS IN A GEOJSON GROUPED TOGETHER
import geopandas
import pandas as pd

gdf = geopandas.read_file("field_poygons_eastwest.json")

# gdf.iloc[0]['field_attributes']['CropType']

gdf= gdf.to_crs({'init': 'epsg:3857'})
gdf['CropType'] = gdf['field_attributes'].apply(pd.Series)['CropType']

gdf = gdf[gdf['CropType']!='Tomato']
gdf['Area'] = gdf['geometry'].area
# gdf.head()

gdf.groupby(['CropType'])['Area'].sum()

C:\Users\USER\.conda\envs\ml4eo\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CropType
Cabbage         154
Cucumber          1
Eggplant         74
Okra              9
Onion            76
Pumpkin          45
Sweet Pepper     18
Watermelon       68
Name: field_id, dtype: int64

In [89]:
import geopandas
import pandas as pd
from tqdm import tqdm
import numpy as np
import json

sentinel_df = pd.read_csv("east_west_sentinel_data.csv")
precip_df = pd.read_csv("east_west_precipitation_data.csv")
temp_df = pd.read_csv("east_west_temperature_data.csv")

months = [
    "january", "february", "march", "april", "may", "june",
    "july", "august", "september", "october", "november", "december"
]

df_rows = []
for index, row in tqdm(sentinel_df.iterrows()):
    df_row = {}

    year_month = int(row["system:index"].split("_")[0])
    if year_month < 11:
        df["year"] = 2020
    else:
        df["year"] = 2021
    df_row["mot"]
    # df_row["system:index"] = row["system:index"]
    df_row["field_id"] = row["field_id"]
    precip_row = precip_df.loc[precip_df["system:index"] == row["system:index"]]
    temp_row = temp_df.loc[temp_df["system:index"] == row["system:index"]]
    df_row["field_ndvi"] = row["NDVI_mean"]
    df_row["field_ndwi"] = row["NDWI_mean"]
    # print("\n", precip_row["mean"].iloc[0].item(), "\n\n", type(precip_row["mean"].iloc[0].item()))
    df_row["field_rainfall"] = precip_row.iloc[0]["mean"].item() if not precip_row.empty else None
    df_row["field_temperature"] = temp_row.iloc[0]["mean"].item() if not temp_row.empty else None
    print("\n\n",df_row, type(df_row["field_temperature"]))
    df_rows.append(df_row)
    break

df2 = pd.DataFrame(df_rows)
df2.to_csv("random_data.csv", index=False)
df2.head()
# print(sentinel_df.columns)
# precip_df.columns
# print(
#     len(sentinel_df.index), len(precip_df.index)
# )

0it [00:00, ?it/s]

 {'system:index': '0_00000000000000000015', 'field_id': 'e00d7e2a-7ebb-4b33-946b-d5f7961d954c', 'field_ndvi': 0.287836492061615, 'field_ndwi': 0.2380281686782837, 'field_rainfall': 2.4969465732574463, 'field_temperature': 299.2894897460937} <class 'float'>



### Dataset Description

This dataset contains field boundaries for crop fields in Kenya as well as the crop types present. Each field can have multiple crops present and are represented in the "Crop1", "Crop2", "Crop3", "Crop4", and "Crop5" columns. Each record also has the date the survey was conducted, when the crops were planted, an estimated harvest date, and whether the planting date is an estimate.

### Listing all of the different types of crops present in this dataset

In [2]:
list(pd.unique(data[['Crop1', 'Crop2', 'Crop3', 'Crop4', 'Crop5']].values.ravel('K')))

['Cassava',
 'Maize',
 'Sorghum',
 'Bean',
 'Groundnut',
 'Fallowland',
 'Millet',
 'Tomato',
 'Sugarcane',
 'Sweetpotato',
 'Banana',
 '',
 'Cowpea',
 'Soybean']

### For you to do

In [3]:
# Write your code to complete the following functions

def get_crop_area(crop):
    # Calculate the total area in square meters of the fields which contain the selected crop
    selected_fields = data.loc[
        (data['Crop1'] == crop) | (data['Crop2'] == crop) |
        (data['Crop3'] == crop) | (data['Crop4'] == crop) | (data['Crop5'] == crop)
    ]
    # print(f"Number of fields with this crop are {len(selected_fields.index)}")
    # print(f"\nTotal number of fields are {len(data.index)}")
    # # This is already in squeare meters. According to the GIS stackexchange link in this cell
    selected_fields['area (sqm)'] = selected_fields['geometry'].area
    # # print(selected_fields.area.round(2).head())
    # print(f"\nTotal area for selected fields is {int(selected_fields.area.sum())}")
    
    #TODO Figure out which coordinate system was used.
    # Seems like its already in a coordinate system, basing on this -> https://gis.stackexchange.com/a/218453
    
    # WRITE YOUR CODE HERE
    area = int(selected_fields.area.sum())
    return area

get_crop_area("Bean")

4892

In [4]:
# Run this to validate your function works correctly

assert int(get_crop_area('Maize')) == 192534, "Sorry wrong answer"
assert int(get_crop_area('Sorghum')) == 227, "Sorry wrong answer"
assert int(get_crop_area('Banana')) == 946, "Sorry wrong answer"
print("Congratulations, all is working just fine !!!")

Congratulations, all is working just fine !!!


### Extra Bonus questions

In [12]:
import fiona
import rasterio
import rasterio.mask
import pycrs


def masked_raster(input_file, raster_file):
    # Create a masked version of the input raster where pixels falling within one of the fields are set to `1` and pixels outside the fields are set to `0`
    
    # WRITE YOUR CODE HERE

    out_img = rasterio.open(raster_file).read()
    return out_img

def reproject_raster(raster_file, dst_crs):
    # Reproject the input raster to the provided CRS
    
    src = rasterio.open(raster_file)
    
    with rasterio.open(raster_file) as src:
        transform, width, height = rasterio.warp.calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })  
        
        split_name = raster_file.split(".")[0]
        out_file_name =f"{split_name}_projected.tif"
        with rasterio.open(out_file_name, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                rasterio.warp.reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=rasterio.warp.Resampling.nearest)
                
        # dst = src
        return dst


In [13]:
str(reproject_raster('crops.tif', 'EPSG:4326').crs)

'EPSG:4326'

In [14]:
# Run this to validate your function works correctly
# print(masked_raster('crops.geojson', 'crops.tif')[0].sum())
# assert masked_raster('crops.geojson', 'crops.tif')[0].sum() == 1144636.0, "Sorry wrong answer"
assert str(reproject_raster('crops.tif', 'EPSG:4326').crs) == 'EPSG:4326', "Sorry wrong answer"
print("Congratulations, all is working just fine !!!")

AssertionError: Sorry wrong answer